In [15]:
import numpy as np
from collections import Counter
from nltk.book import *

In [14]:
# text2 = sense and sensibility by jane austen
unigram_counts = Counter(text2)
bigrams = [(text2[i],text2[i+1]) for i in range(len(text2)-1)]
bigram_counts = Counter(bigrams)

In [19]:
def compute_ngram_probability(ngram):
    return bigram_counts[ngram]/unigram_counts[ngram[0]]

In [24]:
def compute_sentence_probability(s):
    s = s.split(' ')
    return np.exp(np.sum([np.log(compute_ngram_probability((s[i],s[i+1]))) for i in range(len(s)-1)]))

In [26]:
compute_sentence_probability('she did not know')

0.00035411344507094833